### $$ Analystes :\;\;\;\;\;\;\;\;\;\;\; Jeff \; Lwear \; DUVERSEAU \;et \;Lionel \;PETIOTE$$

# Project Tittle :

# **$$\ Credit\ Card \ Fraud\ Detection\ Using\ Supervised\ Machine\ Learning \ $$**

# 1. Overview

#### Ce projet vise à concevoir un modèle prédictif capable d’identifier les transactions frauduleuses effectuées par carte de crédit à l’aide de techniques d’apprentissage supervisé. Le jeu de données comprend plus d’un million de transactions issues de cartes authentiques et compromises, chacune contenant un historique détaillé. L’objectif est de mettre en place un système de classification performant permettant de détecter en temps quasi réel les anomalies et de réduire les pertes financières associées à la fraude.

## 1.1. Business Understanding

#### La fraude par carte de crédit représente un défi majeur pour les institutions financières, tant sur le plan économique que réputationnel. Elle se caractérise par une faible proportion de cas frauduleux et par l’évolution constante des méthodes de fraude, ce qui complique leur détection. Ce projet s’inscrit dans une démarche d’aide à la décision visant à automatiser la détection de comportements suspects grâce à l’analyse de données transactionnelles et à l’intelligence artificielle.

 ## 1.2. Objectifs
 
 ### 1.2.1. Objectif général
 
#### Développer un modèle d’apprentissage supervisé capable de prédire avec précision le caractère frauduleux ou non d’une transaction par carte de crédit à partir des données historiques.

### 1.2.2. Objectifs spécifiques

#### Identifier et sélectionner les variables les plus discriminantes permettant de différencier les comportements normaux des comportements suspects.

#### Entraîner, tester et comparer plusieurs modèles de classification afin de retenir celui offrant la meilleure performance pour la détection de fraude.

## 2. Importations
## 2.1. Importation des bibliothèques

In [3]:
# Manipulation des données
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

# Gestion des avertissements
import warnings
warnings.filterwarnings('ignore')

# Options d’affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

## 2.1. Importation des données

In [69]:
# Fichiers de cartes authentiques
gdc1 = pd.read_csv("IDXDataset_gdc01.csv")
gdc2 = pd.read_csv("IDXDataset_gdc02.csv")
gdc3 = pd.read_csv("IDXDataset_gdc03.csv")

# Fichier des cartes compromises
wpc = pd.read_csv("IDXDataset_wpc.csv")

# Fichier des indices de fraude
fraud_idx = pd.read_csv("fraudidx.csv")

# Vérification rapide
print("Données importées avec succès !")
print(f"gdc1 : {gdc1.shape}")
print(f"gdc2 : {gdc2.shape}")
print(f"gdc3 : {gdc3.shape}")
print(f"wpc  : {wpc.shape}")

Données importées avec succès !
gdc1 : (350000, 8)
gdc2 : (350000, 8)
gdc3 : (348575, 8)
wpc  : (318242, 7)


In [45]:
# Aperçu détaillés des différents fichiers
print("La structure de la table gdc1 est la suivante :")
gdc1.head()

La structure de la table gdc1 est la suivante :


,Unnamed: 0,SEQ,Trans_Calendar_Date,Local_Transaction_Time,CD_TYPE,dollar_amt,CTR,CASE_ID
0,1,41,24-Mar-18,15:54:06,GDC,44.20,1,0
1,2,40,26-Mar-18,11:06:31,GDC,57.78,2,0
2,3,39,27-Mar-18,13:03:02,GDC,37.30,3,0
3,4,38,4-Apr-18,13:35:02,GDC,88.32,4,0
4,5,37,4-Apr-18,18:53:21,GDC,46.74,5,0


In [46]:
# Aperçu détaillés des différents fichiers
print("La structure de la table gdc2 est la suivante :")
gdc2.head()

La structure de la table gdc2 est la suivante :


,Unnamed: 0,SEQ,Trans_Calendar_Date,Local_Transaction_Time,CD_TYPE,dollar_amt,CTR,CASE_ID
0,350001,17,29-May-18,14:08:24,GDC,49.29,350001,8536
1,350002,16,1-Jun-18,11:19:27,GDC,67.67,350002,8536
2,350003,15,6-Jun-18,5:53:43,GDC,57.85,350003,8536
3,350004,14,9-Jun-18,6:37:50,GDC,51.09,350004,8536
4,350005,13,13-Jun-18,5:22:14,GDC,47.16,350005,8536


In [47]:
# Aperçu détaillés des différents fichiers
print("La structure de la table gdc3 est la suivante :")
gdc3.head()

La structure de la table gdc3 est la suivante :


,Unnamed: 0,SEQ,Trans_Calendar_Date,Local_Transaction_Time,CD_TYPE,dollar_amt,CTR,CASE_ID
0,700001,34,7-Dec-17,11:11:10,GDC,34.15,700001,17073
1,700002,33,15-Dec-17,15:48:05,GDC,33.00,700002,17073
2,700003,32,27-Dec-17,9:40:44,GDC,33.48,700003,17073
3,700004,31,5-Jan-18,12:59:46,GDC,35.41,700004,17073
4,700005,30,15-Jan-18,18:23:28,GDC,38.46,700005,17073


In [48]:
# Aperçu détaillés des différents fichiers
print("La structure de la table wpc est la suivante :")
wpc.head()

La structure de la table wpc est la suivante :


,SEQ,Trans_Calendar_Date,Local_Transaction_Time,CASE_ID,WP_YN,CD_TYPE,dollar_amt
0,41,25-Apr-17,6:27:00,176765465.0,0,WPC,57.01
1,40,26-Apr-17,17:07:00,176765465.0,0,WPC,57.81
2,39,28-Apr-17,5:05:00,176765465.0,0,WPC,39.95
3,38,1-May-17,7:43:37,176765465.0,0,WPC,58.18
4,37,2-May-17,15:09:06,176765465.0,0,WPC,58.35


### $Commentaire$

#### Compréhension des données

Les fichiers `gdc1`, `gdc2` et `gdc3` contiennent des transactions légitimes effectuées à partir de **25 575 cartes de crédit**, chacune enregistrant **41 transactions** (soit environ 1 048 575 lignes). Chaque carte est identifiée par un `CASE_ID` unique répété 41 fois, et les transactions sont ordonnées selon la séquence `SEQ` (de 41 à 1). Les colonnes `CTR` et `Unnamed:0` ne sont que des identifiants techniques sans utilité analytique.

Le fichier `wpc` regroupe **7 762 cartes compromises**, également dotées de 41 transactions chacune (≈ 318 242 lignes). Contrairement aux fichiers GDC, il inclut une colonne `WP_YN` qui précise la nature de la transaction : **1 pour une transaction frauduleuse (toujours la plus récente, SEQ=1)** et 0 sinon. L’ordre de ses colonnes diffère légèrement, mais la structure reste compatible.

###### Les fichier `gdc1`, `gdc2` et `gdc3` présentent la même structure et peuvent du coup se fusionner sans difficulté.

In [55]:
# Fusion initiale
Data_card = pd.concat([gdc1, gdc2, gdc3], ignore_index=True)
Data_card.shape

(1048575, 8)

###### Les colonnes Unnamed.0 et CTR donnent la même information, il s'agit d'un indentifiant de transaction et du coup ne présentent pas de réelle utilité analytique. Donc, on peut se permettre de les supprimer.

In [56]:
# Suppression des colonnes Unnamed:0 et CTR
Data_card = Data_card.drop(['Unnamed: 0', 'CTR'], axis=1)

(1048575, 6)

In [58]:
# Comparer les colonnes entre les DataFrames
print("Colonnes gdc1:", set(Data_card.columns))
print("Colonnes wpc :", set(wpc.columns))

# Colonnes manquantes ou différentes
print("\nColonnes manquantes dans wpc :", set(Data_card.columns) - set(wpc.columns))
print("Colonnes supplémentaires dans wpc :", set(wpc.columns) - set(Data_card.columns))

Colonnes gdc1: {'CASE_ID', 'CD_TYPE', 'Local_Transaction_Time', 'dollar_amt', 'SEQ', 'Trans_Calendar_Date'}
Colonnes wpc : {'WP_YN', 'CASE_ID', 'CD_TYPE', 'Local_Transaction_Time', 'dollar_amt', 'SEQ', 'Trans_Calendar_Date'}

Colonnes manquantes dans wpc : set()
Colonnes supplémentaires dans wpc : {'WP_YN'}


In [71]:
# Harmoniser les colonnes
common_cols = [col for col in Data_card.columns if col in wpc.columns]
wpc = wpc[common_cols + ['WP_YN']] if 'WP_YN' in wpc.columns else wpc[common_cols]

# Création de la variable cible 'fraude'
wpc['fraude'] = wpc['WP_YN'].apply(lambda x: 1 if x == 1 else 0)
Data_card['fraude'] = 0

# Supprimer la colonne WP_YN
wpc_clean = wpc.drop(columns=['WP_YN'], errors='ignore')

# Fusion globale
Data_card_complete = pd.concat([Data_card, wpc_clean], ignore_index=True)

# Vérifications de cohérence
print("Dimensions :", Data_card_complete.shape)
print("Répartition des classes :")
print(Data_card_complete['fraude'].value_counts())
print("\nAperçu :")
Data_card_complete.head()

Dimensions : (1366817, 7)
Répartition des classes :
0    1359055
1       7762
Name: fraude, dtype: int64

Aperçu :


,SEQ,Trans_Calendar_Date,Local_Transaction_Time,CD_TYPE,dollar_amt,CASE_ID,fraude
0,41,24-Mar-18,15:54:06,GDC,44.20,0.0,0
1,40,26-Mar-18,11:06:31,GDC,57.78,0.0,0
2,39,27-Mar-18,13:03:02,GDC,37.30,0.0,0
3,38,4-Apr-18,13:35:02,GDC,88.32,0.0,0
4,37,4-Apr-18,18:53:21,GDC,46.74,0.0,0
